# Twitter API testing

In [1]:
import tweepy
import time

In [2]:
CONSUMER_KEY = 'P8uR3oU2eTF1hZw2SX1lma8Zw'
CONSUMER_SECRET = 'Kg85hTJdliuTdFroydQvoRsg7cpr5WZm2MdEQtCz8EjcjG04dp'
ACCESS_TOKEN = '386339280-5sy1Smvnkw91cBObIwu3ju6aMQJU4B8X8HZNsDSo'
ACCESS_TOKEN_SECRET = 'WuZNLBRfzZgT6DIDnZOPqaVlt8uMtrvOUJ9aSbUwkmocD'

auth = tweepy.OAuth1UserHandler(
  CONSUMER_KEY, 
  CONSUMER_SECRET, 
  ACCESS_TOKEN, 
  ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth, wait_on_rate_limit=True)

### Data we need to retrieve given a user profile

1. Number of followers
2. Number of original tweets and number of retweets (last 7 days)

For each original tweet:

3. Number of retweets from other users (shares)
4. Number of favorites from other users (likes)
5. Tweet text
6. Replies from other users to user tweet (text)

7. Gather all the data for future SPI computation

We will take Elon Musk user "@JFCadavid" as an example

The API has some limitations:
* We will only be able to retrieve tweets from the last 7 days.
* There is a rate limit of 60req/min. Going beyond will break the Twitter API (i.e. will block temporarily our access tokens)

Having 'RT @user:' as the beginning of a tweet, tells me that this is a retweet

1. Get number of followers

In [17]:
username = "Cristiano"

In [18]:
n_followers = api.search_users(username)[0].followers_count

2. Get the number of tweets (Including retweets, last 7 days)

In [21]:
extracted_tweets = []

for status in tweepy.Cursor(api.search_tweets, 
                            f"from:{username}",
                            count=5).items(10):
    extracted_tweets.append(status)

In [22]:
retweets = [tweet for tweet in extracted_tweets if "RT @" in tweet.text]
tweets = set(extracted_tweets) - set(retweets)
n_retweets = len(retweets)
n_tweets = len(tweets)

print(n_retweets, n_tweets)

0 10


For each original tweet: 3. Get the number of retweets from another users, 4. get the number of favorites from other users (likes) and 5. get the tweet's text

In [23]:
# We will also store the id
n_id_ls = []
n_retweets_ls = []
n_favorites_ls = []
tweet_text_ls = []
for tweet in tweets:
    n_id_ls.append(tweet.id)
    n_retweets_ls.append(tweet.retweet_count)
    n_favorites_ls.append(tweet.favorite_count)
    tweet_text_ls.append(tweet.text)

In [24]:
tweet_text_ls

['https://t.co/OZJAW7HSJM',
 'How many did you get right? Make sure you have the free @livescore app so you can follow all the action, stats and… https://t.co/c8kJhx2CHK',
 'Boas energias, boas sensações, o mesmo empenho e a mesma concentração de que colocamos em cada desafio. Foco! Vonta… https://t.co/fIhve9oJ1p',
 'Prestes a iniciarmos a nossa campanha na maior competição do Mundo. Uma aventura que desejamos longa e repleta de s… https://t.co/CDPIx99Axf',
 'My first ever NFT collection is live now on @Binance. Don’t miss out! \nSign up now with code RONALDO to explore the… https://t.co/Npzu0MFwUm',
 'Victory is a State of Mind. A long tradition of crafting trunks photographed by @annieleibovitz for @LouisVuitton https://t.co/0TsieZP40P',
 'Taking over Times Square with my new wax figure and showing Portugal to the world! \n\n#madametussaudsusa… https://t.co/MMZxrc1FYN',
 'I can’t leave for Qatar without my #Theragun PRO!\nIt’s important to keep my routine consistent and I know I can 

5. Get the replies from other users to user tweets

In [37]:
def get_replies(user_name, tweet_id, max_replies=10, max_attempts=20):
    replies = tweepy.Cursor(api.search_tweets, q='to:{}'.format(user_name),
                                    since_id=tweet_id, tweet_mode='extended').items()

    replies_ls = []

    counter_fetched_rep = 0
    counter_attempts = 0
    while counter_fetched_rep < max_replies and counter_attempts < max_attempts:
        try:
            reply = replies.next()
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if reply.in_reply_to_status_id == tweet_id:
                replies_ls.append(reply.full_text)
                counter_fetched_rep = counter_fetched_rep + 1 
            counter_attempts = counter_attempts + 1 

        except StopIteration:
            break

        except Exception as e:
            print("Failed while fetching replies {}".format(e))
            break
    return replies_ls

In [38]:
replies_ls = [get_replies(username, tweet_id) for tweet_id in n_id_ls]

In [39]:
tweet_text_ls

['https://t.co/OZJAW7HSJM',
 'How many did you get right? Make sure you have the free @livescore app so you can follow all the action, stats and… https://t.co/c8kJhx2CHK',
 'Boas energias, boas sensações, o mesmo empenho e a mesma concentração de que colocamos em cada desafio. Foco! Vonta… https://t.co/fIhve9oJ1p',
 'Prestes a iniciarmos a nossa campanha na maior competição do Mundo. Uma aventura que desejamos longa e repleta de s… https://t.co/CDPIx99Axf',
 'My first ever NFT collection is live now on @Binance. Don’t miss out! \nSign up now with code RONALDO to explore the… https://t.co/Npzu0MFwUm',
 'Victory is a State of Mind. A long tradition of crafting trunks photographed by @annieleibovitz for @LouisVuitton https://t.co/0TsieZP40P',
 'Taking over Times Square with my new wax figure and showing Portugal to the world! \n\n#madametussaudsusa… https://t.co/MMZxrc1FYN',
 'I can’t leave for Qatar without my #Theragun PRO!\nIt’s important to keep my routine consistent and I know I can 

In [40]:
replies_ls

[['@Cristiano Bye Bye United 😂😂',
  '@Cristiano Gabung persinga ngawi bang do!!'],
 [],
 [],
 ['@Cristiano ❤️❤️❤️',
  '@Cristiano Ya queremos verte cabrón! 👊🏻',
  '@Cristiano Good luck！',
  '@Cristiano good luk don  off the king💙💛🇸🇦❤️',
  '@Cristiano Timão lhe espera.',
  '@Cristiano Esa es la actitud 💪🔥',
  '@Cristiano Anyone who attends the Portugal game, I want him to take a picture for me from the stadium \U0001f979\U0001f979\U0001f979 I could not fulfill my dream of watching my legend from the stands. I apologize to you cr7\U0001f979#CR7𓃵  #Portugal  #البرتغال_غانا',
  '@Cristiano BORA ROBOZÃO. Queremos vocês na final contra o Brasil!',
  '@Cristiano Cristiano Ronaldo deve ter tirado uns 30k só de FGTS',
  '@Cristiano اتمنى لك التوفيق ونجاح نحن معك وسوف نبقى معك وفي جانبك نحن البرتغال'],
 [],
 [],
 [],
 [],
 [],
 []]

7. Gather all the data for future SPI computation

In [41]:
from datetime import date
import numpy as np

user1_data = {
    "user_name": username,
    "n_followers": n_followers,
    "n_retweets": n_retweets,
    "n_tweets": n_tweets,
    "n_retweets_to_user": np.sum(n_retweets_ls),
    "n_favorites_to_user": np.sum(n_favorites_ls),
    "n_replies_to_user": np.sum([len(replies_post) for replies_post in replies_ls]),
    "tweets_text": tweet_text_ls,
    "tweets_replies": replies_ls,
    "date": date.today().strftime("%d/%m/%Y")
}

In [42]:
date.today().strftime("%d/%m/%Y")

'24/11/2022'

In [149]:
import json
with open("sample_data.json", "w") as outfile:
    json.dump(user1_data, outfile)

In [43]:
user1_data

{'user_name': 'Cristiano',
 'n_followers': 105288060,
 'n_retweets': 0,
 'n_tweets': 10,
 'n_retweets_to_user': 387561,
 'n_favorites_to_user': 3808870,
 'n_replies_to_user': 12,
 'tweets_text': ['https://t.co/OZJAW7HSJM',
  'How many did you get right? Make sure you have the free @livescore app so you can follow all the action, stats and… https://t.co/c8kJhx2CHK',
  'Boas energias, boas sensações, o mesmo empenho e a mesma concentração de que colocamos em cada desafio. Foco! Vonta… https://t.co/fIhve9oJ1p',
  'Prestes a iniciarmos a nossa campanha na maior competição do Mundo. Uma aventura que desejamos longa e repleta de s… https://t.co/CDPIx99Axf',
  'My first ever NFT collection is live now on @Binance. Don’t miss out! \nSign up now with code RONALDO to explore the… https://t.co/Npzu0MFwUm',
  'Victory is a State of Mind. A long tradition of crafting trunks photographed by @annieleibovitz for @LouisVuitton https://t.co/0TsieZP40P',
  'Taking over Times Square with my new wax figure